# Gides markdown generator
Adopted from Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `guides.py`. Run either from the `markdown_generator` folder after replacing `guides.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: title, author, and paper_url, with a header at the top.

- `author` can be blank, but the other must have vlaues.

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [160]:
!cat guides.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [161]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [162]:
guides = pd.read_csv("guides.tsv", sep="\t", header=0)
guides


,id,title,venu,url_slug,paper_url
0,1,Brigade Organizer's Playbook,Code for America.org,code-for-America-playbook,http://eipapa.github.io/hack-research-mmistake...
1,2,Hackathon Playbook,Hackathons 101,hackathon-playbook-external,http://eipapa.github.io/hack-research-mmistake...
2,3,Health Hackathon Handbook,MIT Hacking Medicine,health-hackathon-handbook,http://eipapa.github.io/hack-research-mmistake...
3,4,@Civicwhitaker Anthology,Christopher Whitaker edited by Daniel X. O'Neil,civicwhitaker-anthology,http://eipapa.github.io/hack-research-mmistake...
4,5,The complete guide to organizing a successful ...,hackerearth,hackerearth,http://eipapa.github.io/hack-research-mmistake...


In [163]:
for row, item in guides.iterrows():
    print(item.id)

1
2
3
4
5


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [164]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [165]:
import os
for row, item in guides.iterrows():
    
    md_filename = str(item.id) + "-" + item.url_slug + ".md"
    html_filename = str(item.id) + "-" + item.url_slug
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += """collection: guides"""
    md += """\npermalink: /guide/""" + html_filename
    md += "\nvenue: '" + html_escape(item.venu) + "'"
    #md +="\npaper_url: '" + item.paper_url + "'"
    md += "\n---"
    
    ## Markdown description for individual page
        
#     if len(str(item.paper_url)) > 5:
#         md += "\n[<span style=\"color: #c41e3a\">Download PDF here.</span>](" + item.paper_url + ")\n"
    md += "\n[Download PDF here.](" + item.paper_url + ")\n"
#     if len(str(item.excerpt)) > 5:
#         md += "\n**Abstract**:" + html_escape(item.excerpt) + "\n"    
#     md += "\nAbstract: " + html_escape(item.description) + "\n"
#     md += "\n**Recommended citation**: " + item.citation
    md_filename = os.path.basename(md_filename)
    with open("../_guides/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [166]:
!ls ../_guides/

1-code-for-America-playbook.md   4-civicwhitaker-anthology.md
2-hackathon-playbook-external.md 5-hackerearth.md
3-health-hackathon-handbook.md


In [ ]:
!cat ../_guides/2009-10-01-paper-title-number-1.md